In [ ]:
cf_file = snakemake.input.cf_file
grid_areas = snakemake.input.grid_areas

cf_out_file = snakemake.output.capfacfile
out_areassolar = snakemake.output.areassolar
out_areaswindon = snakemake.output.areaswindonshore
out_areaswindoff = snakemake.output.areaswindoffshore
out_areaswindoff_floating = snakemake.output.areaswindoffshore_floating

out_dists = snakemake.output.vre_connection_dists

date_range = snakemake.params.date_range
year = snakemake.wildcards.year

date_range = [year + "-" + date for date in date_range]

In [ ]:
import xarray as xr
import pandas as pd
import geopandas as gpd
import numpy as np

In [ ]:
gridcellnamingfunctionxr = (
    lambda da: "x"
    + (da.x * 100).astype("int").astype("str").astype("object")
    + "y"
    + (da.y * 100).astype("int").astype("str").astype("object")
)

In [ ]:
areas = pd.read_csv(grid_areas)

areas

In [ ]:
ds = xr.open_dataarray(cf_file)

ds

In [ ]:
import time

In [ ]:
cf_out = []

for tech in np.unique(ds["Tech"]):
    start_time = time.time()

    cf_grid = ds.sel(Tech=tech, time=slice(date_range[0], date_range[1]))

    ntime = len(cf_grid.time)

    cf_grid = (
        cf_grid.stack(spatial=["x", "y"])
        .assign_coords(gridcell=gridcellnamingfunctionxr)
        .set_index(spatial="gridcell")
        .drop_vars(["lat", "lon", "Tech"])
        .assign_coords(time=range(0, ntime))
        .transpose("spatial", "time")
        .to_pandas()
    )

    print(time.time() - start_time)

    start_time = time.time()

    cf_grid = areas.loc[
        areas["Tech"] == tech, ["zone", "index", "gridcell", "area","dist"]
    ].merge(cf_grid, how="inner", left_on=["gridcell"], right_on=["spatial"])

    print(time.time() - start_time)

    start_time = time.time()

    df = cf_grid[["zone", "index", "gridcell", "area", "dist"]]

    df.loc[:,"dist"]=df["dist"]*df["area"]
    cf = cf_grid.loc[:, np.arange(ntime)].values * cf_grid["area"].values.reshape(-1, 1)

    cf_grid = pd.concat((df, pd.DataFrame(cf, columns=np.arange(ntime))), axis=1)

    cf_grid = cf_grid.drop("gridcell", axis=1)

    groups = cf_grid.groupby(["zone", "index"])

    
    cf_grid = (
        groups.sum()[["dist"]+list(np.arange(ntime))]
        .div(groups["area"].sum(), axis=0)
        .assign(area=groups["area"].sum())
        .reset_index()
        .melt(id_vars=["zone", "index", "area","dist"])
        .rename({"variable": "time", "index": "spatial", "value": 0}, axis=1)
        )

    print(time.time() - start_time)

    cf_grid["technology"] = tech

    cf_grid = cf_grid[["zone", "time", "technology", "spatial", "area", "dist", 0]]

    cf_out.append(cf_grid)

In [ ]:
out_cf = pd.concat(cf_out).round(2)

In [ ]:
out_areas = (
    out_cf.drop_duplicates(subset=["technology", "zone", "spatial"])
    .drop(columns=["time", 0])
    .set_index(["zone","spatial"]))

if snakemake.params.include_rooftop:
    
    rooftop_pv=pd.read_csv(snakemake.input.rooftop_areas).set_index(["zone","spatial"])

    if snakemake.wildcards.spatial=="region":
        rooftop_pv=rooftop_pv.groupby("zone").sum()
        rooftop_pv["spatial"]=rooftop_pv.index
        rooftop_pv=rooftop_pv.reset_index().set_index(["zone","spatial"])
    
    out_areas.loc[out_areas.technology == "Solar","area"]+=rooftop_pv.squeeze()

out_areas = (out_areas.reset_index()
            .assign(
                new_idx=lambda x: x["technology"] + "." + x["zone"] + "." + x["spatial"],
            )
            .set_index("new_idx")
            .drop(columns=["spatial"])
            .query("area != 0")
            )

# write out connection distances
# for few cases where lower spatial level added back in for rooftop solar 
# it has no distance (not even 0) so add back in with fillna

(
    out_areas.round(1)
    .loc[:,["dist"]]
    .fillna(0.0)
    .to_csv(out_dists)
)

(
    out_areas.round(1)
    .loc[out_areas["technology"] == "Solar", ["area"]]
    .to_csv(out_areassolar, header=False, sep=" ")
)

(
    out_areas.round(1)
    .loc[out_areas["technology"] == "Windonshore", ["area"]]
    .to_csv(out_areaswindon, header=False, sep=" ")
)

(
    out_areas.round(1)
    .loc[out_areas["technology"] == "Windoffshore", ["area"]]
    .to_csv(out_areaswindoff, header=False, sep=" ")
)

if snakemake.params.windoff_floating:
    (
        out_areas.round(1)
        .loc[out_areas["technology"] == "WindoffshoreFloat", ["area"]]
        .to_csv(out_areaswindoff_floating, header=False, sep=" ")
    )

out_cf[["time", "technology", "spatial", 0]].to_csv(cf_out_file, index=False)

# (out_cf[["time","technology","spatial",0]].pivot_table(index=["spatial","technology"],columns="time")
#     .to_csv("D:\\science\\models\\highRES\\model_versions\\highRES-Europe-WF\\work\\models\\2010\\nuts2\\cf.csv"))